In [1]:
import pandas as pd
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_absolute_error, mean_squared_error
from tqdm import tqdm
import warnings

# Suppress warnings
warnings.filterwarnings("ignore")

In [2]:
kaggle = False

if kaggle:
    path = ""
else:
    path = "Week8_Data_Sales"

train = pd.read_csv(f"{path}/train.csv", parse_dates= ['date'])
test = pd.read_csv(f"{path}/test.csv", parse_dates= ['date'])

## Prepare the data

In [3]:
# Aggregate sales at daily level per store and family
train_grouped = train.groupby(["store_nbr", "family", "date"]).agg({"sales": "sum", "onpromotion": "sum"}).reset_index()

# Ensure datetime index for time series forecasting
train_grouped["date"] = pd.to_datetime(train_grouped["date"])

In [4]:
# Temporal train-test split (80% train, 20% test based on time)
split_date = train_grouped["date"].quantile(0.8)  # 80% cutoff date
train_set = train_grouped[train_grouped["date"] <= split_date]
test_set = train_grouped[train_grouped["date"] > split_date]

In [5]:
train_set

,store_nbr,family,date,sales,onpromotion
0,1,AUTOMOTIVE,2013-01-01,0.0,0
1,1,AUTOMOTIVE,2013-01-02,2.0,0
2,1,AUTOMOTIVE,2013-01-03,3.0,0
3,1,AUTOMOTIVE,2013-01-04,3.0,0
4,1,AUTOMOTIVE,2013-01-05,5.0,0
...,...,...,...,...,...
3000547,54,SEAFOOD,2016-09-08,0.0,0
3000548,54,SEAFOOD,2016-09-09,0.0,0
3000549,54,SEAFOOD,2016-09-10,4.0,1
3000550,54,SEAFOOD,2016-09-11,0.0,0


## Build The Model

In [6]:
# Fit ARIMA Model for each store-family combination with progress bar
results = []
store_family_groups = train_set.groupby(["store_nbr", "family"])

with tqdm(total=len(store_family_groups), desc="Training ARIMA Models") as pbar:
    for (store, family), df in store_family_groups:
        df = df.set_index("date").sort_index()

        # Ensure the time series has a regular frequency
        df = df.asfreq("D").fillna(method="ffill").fillna(0)  # Fill missing dates

        test_df = test_set[(test_set["store_nbr"] == store) & (test_set["family"] == family)]
        if test_df.empty:
            print(f"Skipping {store}, {family} - No test data available.")
            pbar.update(1)
            continue

        try:
            # Initial ARIMA model
            model = ARIMA(df["sales"], order=(5, 1, 0))  # Default p, d, q values
            fit = model.fit()

        except Exception as e:
            print(f"Retrying {store}, {family} with different ARIMA parameters due to: {e}")
            try:
                # Retry with different parameters
                model = ARIMA(df["sales"], order=(3, 1, 3))
                fit = model.fit()
            except Exception as e:
                print(f"Skipping {store}, {family} after second failure: {e}")
                pbar.update(1)
                continue

        test_df = test_df.set_index("date").sort_index()
        test_df = test_df.asfreq("D").fillna(method="ffill").fillna(0)  # Ensure aligned test set

        preds = fit.forecast(steps=len(test_df))  # Forecast for test period

        # Ensure actual values are not empty
        if len(test_df["sales"].values) == 0 or len(preds) == 0:
            print(f"Skipping {store}, {family} - Empty prediction or actual values.")
            pbar.update(1)
            continue

        # Evaluate model
        mae = mean_absolute_error(test_df["sales"].values, preds)
        rmse = np.sqrt(mean_squared_error(test_df["sales"].values, preds))

        results.append([store, family, mae, rmse])

        pbar.update(1)  # Update progress bar

Training ARIMA Models: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1782/1782 [09:35<00:00,  3.10it/s]


In [7]:
# Convert evaluation results to DataFrame
metrics_df = pd.DataFrame(results, columns=["store_nbr", "family", "MAE", "RMSE"])

In [8]:
metrics_df

,store_nbr,family,MAE,RMSE
0,1,AUTOMOTIVE,2.222553,3.049151
1,1,BABY CARE,0.000000,0.000000
2,1,BEAUTY,2.635572,3.006191
3,1,BEVERAGES,549.477572,646.537964
4,1,BOOKS,0.649852,1.306228
...,...,...,...,...
1777,54,POULTRY,17.301747,25.527164
1778,54,PREPARED FOODS,20.867308,26.280812
1779,54,PRODUCE,148.060217,203.044024
1780,54,SCHOOL AND OFFICE SUPPLIES,1.816024,5.860394


## Kaggle

In [9]:
test = test.sort_values(by=["store_nbr", "family", "date"])

# Ensure datetime index for prediction
test["date"] = pd.to_datetime(test["date"])

In [10]:
# Predict sales for test dataset with progress bar
predictions = []
test_groups = test.groupby(["store_nbr", "family"])

with tqdm(total=len(test_groups), desc="Predicting Sales with ARIMA") as pbar:
    for (store, family), df in test_groups:
        df = df.set_index("date").sort_index()
        df = df.asfreq("D").fillna(method="ffill").fillna(0)  # Ensure daily frequency in test data

        try:
            # Fit model on full training data
            train_series = train_set[(train_set["store_nbr"] == store) & (train_set["family"] == family)]
            if train_series.empty:
                print(f"Skipping {store}, {family} - No training data available.")
                pbar.update(1)
                continue

            train_series = train_series.set_index("date").sort_index()
            train_series = train_series.asfreq("D").fillna(method="ffill").fillna(0)  # Ensure a regular time series

            model = ARIMA(train_series["sales"], order=(5, 1, 0))  # ARIMA (p,d,q)
            fit = model.fit()

            if len(df) == 0:
                print(f"Skipping {store}, {family} - No test data for forecasting.")
                pbar.update(1)
                continue

            preds = fit.forecast(len(df))  # Forecast for test period
            df["sales"] = preds.values
            df = df.reset_index()

            predictions.append(df[["id", "sales"]])

        except Exception as e:
            print(f"Skipping {store}, {family} due to error: {e}")

        pbar.update(1)  # Update progress bar

Predicting Sales with ARIMA: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1782/1782 [14:12<00:00,  2.09it/s]


In [11]:
# Save predictions to CSV
if predictions:
    predictions_df = pd.concat(predictions)
    predictions_df.to_csv("sales_predictions_arima.csv", index=False)
else:
    print("No predictions were generated.")

In [12]:
metrics_df.to_csv("sales_predictions_arima_metrics.csv", index=False)